## Análises sobre correlações e associações dos dados

In [47]:
# importando bibliotecas

# numericas
import numpy as np

# manipulacao de dados
import pandas as pd

# graficas
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# utilidades
# from google.colab import drive

import warnings
warnings.filterwarnings('ignore')

from collections import Counter

In [48]:
df = pd.read_csv('https://raw.githubusercontent.com/ImagineDogs/TranstornosMentais/main/dados/df_cid10.csv')

### Associações

#### Associações entre diagnósticos

### Apiori
O algoritmo apriori é um algoritmo para encontrar padrões frequentes em uma base de dados, aqui está sendo utilizado para investigar se pacientes que possuem alguma comorbidade geralmente possuem outras.

Valor mínimo de suporte(porcentagem de ocorrência na base) escolhido assumindo o valor equiprovável para as variáveis $1/3$, multiplicado por $1/8$, para compreender informação sobre o número de campos a serem preenchidos, e finalmente elevado ao quadrado para obter um valor equiprovável de base para o aparecimento de duas comorbidades ao mesmo tempo. Valor final $(\frac{1}{3} * \frac{1}{8})^2 = 0.00173611111$

In [49]:
from mlxtend.frequent_patterns import apriori

def apriori_comorbidades(df, agg_method='first', min_support=0.00173611111):
  '''
  Função para calcular apiori para as comorbidades dado um critério de agregação
  '''

  comorbidades = ['traumatismo',
 'prob_respiratorios',
 'avc',
 'convulsao',
 'has',
 'gravidez',
 'diabetes',
 'doenca_infecto']

  comorbidades.insert(0,'cod_paciente')

  df_comorbidades = df[comorbidades].copy()
  df_pacientes_comorbidades = df_comorbidades.groupby('cod_paciente').agg(agg_method).reset_index(drop=True)
  df_pacientes_comorbidades = df_pacientes_comorbidades.replace("SIM", True).replace("NAO INFORMADO", "NAO").replace("NAO", False)

  result = apriori(df_pacientes_comorbidades, use_colnames=True, min_support=min_support)
  result['MinSupChange'] = result['support']/min_support
  return result.sort_values('support', ascending=False).reset_index(drop=True)[['itemsets','support','MinSupChange']]

In [50]:
apriori_comorbidades(df,'first')

,itemsets,support,MinSupChange
0,(has),0.087271,50.268249
1,(convulsao),0.045043,25.944902
2,(diabetes),0.043032,24.786648
3,(doenca_infecto),0.026141,15.057309
4,"(has, diabetes)",0.025337,14.594008
5,(traumatismo),0.022723,13.088277
6,(prob_respiratorios),0.014076,8.107782
7,"(has, convulsao)",0.008848,5.096320
8,(avc),0.008043,4.633018
9,"(traumatismo, convulsao)",0.005630,3.243113


In [51]:
apriori_comorbidades(df,'last')

,itemsets,support,MinSupChange
0,(has),0.092298,53.163885
1,(convulsao),0.048060,27.682284
2,(diabetes),0.046652,26.871506
3,(doenca_infecto),0.028755,16.563040
4,"(has, diabetes)",0.027147,15.636437
5,(traumatismo),0.021516,12.393324
6,(prob_respiratorios),0.014076,8.107782
7,(avc),0.009652,5.559622
8,"(has, convulsao)",0.008245,4.748844
9,"(traumatismo, convulsao)",0.004826,2.779811


In [52]:
pacientes_repetidos = list((df['cod_paciente'].value_counts()>1).loc[lambda x : x == True].index)

In [53]:
df_repetidos = df[df['cod_paciente'].isin(pacientes_repetidos)]
df_unicos = df[~df['cod_paciente'].isin(pacientes_repetidos)]

In [54]:
df_repetidos.head()

,Unnamed: 0.1,Unnamed: 0,data_internacao_real,data_cadastro,sexo,data_nascimento,uf,municipio,profissao_ocupacao,cod_paciente,...,cid10_seg_familia_alta,cid10_seg_descricao_familia_alta,cid10_faixa,cid10_descricao_faixa,cid10_seg_faixa,cid10_seg_descricao_faixa,cid10_faixa_alta,cid10_descricao_faixa_alta,cid10_seg_faixa_alta,cid10_seg_descricao_faixa_alta
1,1,1,2012-06-25,2012-08-02,F,1971-02-10,SP,RIBEIRAO PRETO,DESEMPREGADA,40,...,F00-F99,Transtornos mentais e comportamentais,NaN,NaN,NaN,NaN,F10-F19,Transtornos mentais e comportamentaisl devido...,F30-F39,Transtornos de humor
2,2,2,2013-04-12,2012-08-02,F,1971-02-10,SP,RIBEIRAO PRETO,DESEMPREGADA,40,...,F00-F99,Transtornos mentais e comportamentais,F10-F19,Transtornos mentais e comportamentaisl devido...,NaN,NaN,F10-F19,Transtornos mentais e comportamentaisl devido...,F10-F19,Transtornos mentais e comportamentaisl devido...
3,3,3,2014-11-12,2012-08-02,F,1971-02-10,SP,RIBEIRAO PRETO,DESEMPREGADA,40,...,F00-F99,Transtornos mentais e comportamentais,F10-F19,Transtornos mentais e comportamentaisl devido...,F60-F69,Transtornos de personalidade adulta e outros ...,F10-F19,Transtornos mentais e comportamentaisl devido...,F20-F29,"Esquizofrenia, Esquizotípico, Delírio e outro..."
4,4,4,2015-01-15,2012-08-02,F,1971-02-10,SP,RIBEIRAO PRETO,DESEMPREGADA,40,...,F00-F99,Transtornos mentais e comportamentais,F10-F19,Transtornos mentais e comportamentaisl devido...,F10-F19,Transtornos mentais e comportamentaisl devido...,F10-F19,Transtornos mentais e comportamentaisl devido...,F20-F29,"Esquizofrenia, Esquizotípico, Delírio e outro..."
5,5,5,2015-04-23,2012-08-02,F,1971-02-10,SP,RIBEIRAO PRETO,DESEMPREGADA,40,...,NaN,NaN,F10-F19,Transtornos mentais e comportamentaisl devido...,F60-F69,Transtornos de personalidade adulta e outros ...,F10-F19,Transtornos mentais e comportamentaisl devido...,NaN,NaN


In [55]:
df_unicos.head()

,Unnamed: 0.1,Unnamed: 0,data_internacao_real,data_cadastro,sexo,data_nascimento,uf,municipio,profissao_ocupacao,cod_paciente,...,cid10_seg_familia_alta,cid10_seg_descricao_familia_alta,cid10_faixa,cid10_descricao_faixa,cid10_seg_faixa,cid10_seg_descricao_faixa,cid10_faixa_alta,cid10_descricao_faixa_alta,cid10_seg_faixa_alta,cid10_seg_descricao_faixa_alta
0,0,0,2012-07-12,NaN,F,1957-07-30,SP,RIBEIRAO PRETO,DONA DE CASA,39,...,NaN,NaN,NaN,NaN,NaN,NaN,F20-F29,"Esquizofrenia, Esquizotípico, Delírio e outro...",NaN,NaN
9,9,9,2012-06-26,2012-08-02,F,1990-11-29,SP,SANTA ROSA DE VITERBO,NAO INFORMADO,41,...,NaN,NaN,NaN,NaN,NaN,NaN,F10-F19,Transtornos mentais e comportamentaisl devido...,NaN,NaN
10,10,10,2012-07-23,2012-08-02,F,1982-02-05,SP,SANTA RITA DO PASSA QUATRO,NAO INFORMADO,42,...,NaN,NaN,NaN,NaN,NaN,NaN,F10-F19,Transtornos mentais e comportamentaisl devido...,NaN,NaN
21,21,21,2012-07-27,2012-08-02,F,1989-08-22,SP,SERTAOZINHO,NAO INFORMADO,46,...,NaN,NaN,NaN,NaN,NaN,NaN,F10-F19,Transtornos mentais e comportamentaisl devido...,NaN,NaN
22,22,22,2012-07-23,2012-08-02,F,1994-08-07,SP,BATATAIS,NAO INFORMADO,47,...,NaN,NaN,NaN,NaN,NaN,NaN,F10-F19,Transtornos mentais e comportamentaisl devido...,NaN,NaN


In [56]:
apriori_comorbidades(df_repetidos)

,itemsets,support,MinSupChange
0,(has),0.070391,40.545251
1,(convulsao),0.049721,28.639106
2,(diabetes),0.043017,24.777654
3,(doenca_infecto),0.027933,16.089385
4,(traumatismo),0.022905,13.193296
5,"(has, diabetes)",0.022346,12.871508
6,(prob_respiratorios),0.013966,8.044693
7,"(has, convulsao)",0.010056,5.792179
8,(avc),0.006704,3.861453
9,"(traumatismo, convulsao)",0.005587,3.217877


In [57]:
apriori_comorbidades(df_unicos)

,itemsets,support,MinSupChange
0,(has),0.096764,55.736098
1,(diabetes),0.043041,24.791706
2,(convulsao),0.042413,24.429783
3,"(has, diabetes)",0.027019,15.562677
4,(doenca_infecto),0.025134,14.476909
5,(traumatismo),0.022620,13.029218
6,(prob_respiratorios),0.014138,8.143261
7,(avc),0.008797,5.066918
8,"(has, convulsao)",0.008168,4.704995
9,"(has, prob_respiratorios)",0.005969,3.438266


#### Associações temporais entre internaçoes

In [58]:
datas = df.filter(regex=('data')).columns
for data in datas:
  df[data] = pd.to_datetime(df[data])

In [59]:
df['data_internacao_real_ano'] = df['data_internacao_real'].apply(lambda x: x.year)
df['data_internacao_real_mes'] = df['data_internacao_real'].apply(lambda x: x.month)
df['data_internacao_real_mes_ano'] = df['data_internacao_real'].apply(lambda x: str(x.year) + '/' + str(x.month) if len(str(x.month)) > 1 else str(x.year) + '/0' + str(x.month))

ano_internacao = df[['data_internacao_real_ano', 'cod_paciente']].groupby(['data_internacao_real_ano']).count().reset_index().sort_values(by='data_internacao_real_ano')
ano_internacao.columns = ['Ano', 'Quantidade']

ano_mes_internacao = df[['data_internacao_real_mes', 'cod_paciente', 'data_internacao_real_ano']].groupby(['data_internacao_real_ano', 'data_internacao_real_mes']).count().reset_index().sort_values(by='data_internacao_real_mes')
ano_mes_internacao.columns = ['Ano', 'Mes', 'Quantidade']
mes_internacao = ano_mes_internacao[['Mes', 'Quantidade']].groupby(['Mes']).sum().reset_index().sort_values(by='Mes')


data_internacao = df[['data_internacao_real_mes_ano', 'cod_paciente']].groupby(['data_internacao_real_mes_ano']).count().reset_index().sort_values(by='data_internacao_real_mes_ano')
data_internacao.columns = ['Mes Ano', 'Quantidade']

ano_mes_internacao_cum = ano_mes_internacao.loc[ano_mes_internacao['Ano'] != 2012].copy()
for i in range(1, ano_internacao['Ano'].shape[0], 1):
    ano_mes_internacao_cum['Quantidade'].loc[ano_mes_internacao_cum['Ano'] == ano_internacao['Ano'][i]] = ano_mes_internacao['Quantidade'].loc[ano_mes_internacao['Ano'] == ano_internacao['Ano'][i]]/ano_internacao['Quantidade'][i]
mes_internacao_cum = ano_mes_internacao_cum[['Mes', 'Quantidade']].groupby(['Mes']).sum().reset_index().sort_values(by='Mes')



Visualizar a relação entre internações e o tempo para identificar possíveis associações e sazonalidades.

Análise para os próximos dois gráficos: 
Por se tratar de uma janela curta de tempo com bastante variação, isso nos impede de determinar se houve uma crescente ou não nas internações com o passar do tempo. É necessário fazer uma ressalva quanto ao ano de 2012, pois foi o ano que se iniciou os registros, que não foram coletados desde janeiro, portanto será naturalmente menor que os outros anos.

In [60]:
px.line(ano_internacao, x='Ano', y = 'Quantidade', title='Internações por Ano')

In [61]:
px.line(data_internacao, x='Mes Ano', y = 'Quantidade', title='Internações por Mes/Ano')

In [62]:
px.line(ano_mes_internacao, x='Mes', y = 'Quantidade', color='Ano', title='Internações por Mês agrupado por Ano')
px.line(ano_mes_internacao_cum.sort_values(by='Ano'), x='Ano', y = 'Quantidade', color='Mes', title='Internações por Mês agrupado por Ano')

Ao analisar as internações por mês é notável que os meses finais possuem uma maior quantidade de intenações em comparação com os iniciais.

Outra análise pode ser feita ao olhar o valor acumulado da proporção de um mês relativo ao seu próprio ano, já que cada ano tem um total diferente de internações, e assim podemos enxergar proporcionalmente. Para essa análise o ano de 2012 é removido pois ele possui a questão de ser o ano inicial da coleta e pode prejudicar as proporções.

In [63]:
px.line(mes_internacao, x='Mes', y = 'Quantidade', title='Internações por Mês')

In [64]:
px.line(mes_internacao_cum, x='Mes', y = 'Quantidade', title='Proporção acumulada de Internações por Mês')

Ao analisar as internações por mês dado os anos, utilizando a mesma idea de proporção, não é possível encontrar um tipo de padrão sazonal, alguns meses sobem em determinado ano, enquanto descem em outro ano.

In [65]:
px.line(ano_mes_internacao, x='Mes', y = 'Quantidade', color='Ano', title='Internações por Mês agrupado por Ano')

In [66]:
px.line(ano_mes_internacao_cum, x='Mes', y = 'Quantidade', color='Ano', title='Proporção Acumulada de Internações por Mês agrupado por Ano')

### Correlações

#### Correlações temporais entre diagnósticos

In [67]:
df['Ano'] = df['data_internacao_real'].apply(lambda x: x.year)

In [68]:
def contagem_agrupado(df, col1, col2 = None, ren_col1 = None, ren_col2 = None, contador = 'cod_paciente'):
    if col2 == None:
        contagem = df[[col1, 'Ano', contador]].copy()
        
        contagem = contagem[['Ano', col1, contador]].groupby([ 'Ano', col1]).count()
        contagem = contagem.reset_index().rename(columns={contador: 'Quantidade'})
        if ren_col1 is not None:
            contagem = contagem.rename(columns={col1: ren_col1})
    else:
        contagem = df[[col1, col2, 'Ano', contador]].copy()
        contagem = contagem[['Ano', col1, col2, contador]].groupby([ 'Ano', col1, col2]).count()
        contagem = contagem.reset_index().rename(columns={contador: 'Quantidade'})
        if ren_col1 is not None :
            contagem = contagem.rename(columns={col1: ren_col1})
        if ren_col2 is not None:
            contagem = contagem.rename(columns={col2: ren_col2})
    return contagem

def outliers(df):
    Q3 = df['Quantidade'].quantile(.75)
    Q1 = df['Quantidade'].quantile(.25)
    dif = Q3-Q1
    superior = Q3 + 1.5 * dif
    df_outliers= df.loc[df['Quantidade'] >= superior]
    return df_outliers

In [69]:
diagnosticos_ano = contagem_agrupado(df, 'cid10_diagnostico', ren_col1='Diagnostico')
diagnosticos_ano

,Ano,Diagnostico,Quantidade
0,2012,F00,1
1,2012,F02,2
2,2012,F03,1
3,2012,F06,9
4,2012,F07,2
...,...,...,...
277,2017,T49,1
278,2017,X70,1
279,2017,Y20,1
280,2017,Z86,1


Ao analisar a distribuição de diagnósticos por ano, é possível indentificar um padrão de diagnósticos que se destacam, como: F20, F19, F31 e F10.

In [70]:
fig = px.line(diagnosticos_ano, x="Ano", y="Quantidade", color='Diagnostico')
fig.show()
fig =px.box(diagnosticos_ano, x = 'Ano',y='Quantidade', hover_name='Diagnostico')
fig.show()

In [71]:
outliers_gerais = pd.DataFrame()
for year in diagnosticos_ano['Ano'].unique():
    outliers_gerais = pd.concat([outliers_gerais, outliers(diagnosticos_ano.loc[diagnosticos_ano['Ano'] == year])])
    
outliers_ranked_freqs = outliers_gerais[['Diagnostico', 'Quantidade']].groupby(['Diagnostico']).count().reset_index().rename(columns={'Quantidade': 'Frequencia'})
outliers_sum = outliers_gerais[['Diagnostico', 'Quantidade']].groupby(['Diagnostico']).sum().reset_index()
outliers_ranked_freqs = outliers_sum.merge(outliers_ranked_freqs, on='Diagnostico').sort_values(by=['Frequencia', 'Quantidade'])
outliers_ranked_freqs['Label'] = pd.Series(['Geral']*len(outliers_ranked_freqs))
outliers_ranked_freqs

,Diagnostico,Quantidade,Frequencia,Label
7,F29,33,1,Geral
0,F06,59,2,Geral
5,F23,82,2,Geral
6,F25,144,3,Geral
10,F33,357,5,Geral
11,F60,368,5,Geral
2,F14,460,5,Geral
1,F10,700,5,Geral
9,F32,556,6,Geral
8,F31,1021,6,Geral


In [72]:
px.scatter(outliers_ranked_freqs, x="Diagnostico", y="Frequencia", size='Quantidade')

In [73]:
outliers_gerais = pd.DataFrame()
for year in diagnosticos_ano['Ano'].unique():
    outliers_gerais = pd.concat([outliers_gerais, outliers(diagnosticos_ano.loc[diagnosticos_ano['Ano'] == year])])
outliers_gerais.head()
# outliers_ranked_freqs = outliers_gerais[['Diagnostico', 'Quantidade']].groupby(['Diagnostico']).count().reset_index().rename(columns={'Quantidade': 'Frequencia'})
# outliers_sum = outliers_gerais[['Diagnostico', 'Quantidade']].groupby(['Diagnostico']).sum().reset_index()
# outliers_ranked_freqs = outliers_sum.merge(outliers_ranked_freqs, on='Diagnostico').sort_values(by=['Frequencia', 'Quantidade'])
# outliers_ranked_freqs['Label'] = pd.Series(['Geral']*len(outliers_ranked_freqs))
# outliers_ranked_freqs

,Ano,Diagnostico,Quantidade
9,2012,F19,133
10,2012,F20,138
18,2012,F31,88
19,2012,F32,59
40,2013,F10,133


In [74]:
outliers_order_ranked_freqs = pd.DataFrame(columns=['Diagnostico', 'Frequencia'])
for codigo in outliers_gerais['Diagnostico'].unique():
    years = outliers_gerais['Ano'].loc[outliers_gerais['Diagnostico'] == codigo].sort_values().tolist()
    flag = False
    sequencia = 0
    for year in outliers_gerais['Ano'].unique():
        if year in years:
            flag = True
            sequencia += 1
        elif flag:
            break
    if sequencia > 1:
        outliers_order_ranked_freqs = pd.concat([outliers_order_ranked_freqs, pd.DataFrame({'Diagnostico': [codigo], 'Frequencia': [sequencia]})])

outliers_sum = outliers_gerais[['Diagnostico', 'Quantidade']].groupby(['Diagnostico']).sum().reset_index()
outliers_order_ranked_freqs = outliers_sum.merge(outliers_order_ranked_freqs, on='Diagnostico').sort_values(by=['Frequencia', 'Quantidade'])
outliers_order_ranked_freqs['Label'] = pd.Series(['Sequencial']*len(outliers_order_ranked_freqs))
outliers_order_ranked_freqs.head()

,Diagnostico,Quantidade,Frequencia,Label
0,F06,59,2,Sequencial
5,F23,82,2,Sequencial
6,F25,144,3,Sequencial
9,F33,357,5,Sequencial
10,F60,368,5,Sequencial


In [75]:
outliers = pd.DataFrame()
for year in diagnosticos_ano['Ano'].unique():
    q1 = diagnosticos_ano[['Quantidade']].loc[diagnosticos_ano['Ano'] == year].quantile(.25, interpolation='lower')[0]
    q3 = diagnosticos_ano[['Quantidade']].loc[diagnosticos_ano['Ano'] == year].quantile(.75, interpolation='lower')[0]
    iqr = q3 - q1
    limite_superior = q3 + 1.5 * iqr
    outliers = pd.concat([outliers, diagnosticos_ano.loc[diagnosticos_ano['Ano'] == year].loc[diagnosticos_ano['Quantidade'] >= limite_superior]])
    
    outliers_order_ranked_freqs = pd.DataFrame(columns=['Diagnostico', 'Frequencia'])
for codigo in outliers['Diagnostico'].unique():
    years = outliers['Ano'].loc[outliers['Diagnostico'] == codigo].sort_values().tolist()
    flag = False
    sequencia = 0
    for year in outliers['Ano'].unique():
        if year in years:
            flag = True
            sequencia += 1
        elif flag:
            break
    # print(codigo, sequencia)
    if sequencia > 1:
        outliers_order_ranked_freqs = pd.concat([outliers_order_ranked_freqs, pd.DataFrame({'Diagnostico': [codigo], 'Frequencia': [sequencia]})])


outliers_sum = outliers[['Diagnostico', 'Quantidade']].groupby(['Diagnostico']).sum().reset_index()
outliers_order_ranked_freqs = outliers_sum.merge(outliers_order_ranked_freqs, on='Diagnostico').sort_values(by=['Frequencia', 'Quantidade'])
outliers_order_ranked_freqs['Label'] = pd.Series(['Sequencial']*len(outliers_order_ranked_freqs))
outliers_order_ranked_freqs.head()

,Diagnostico,Quantidade,Frequencia,Label
0,F06,59,2,Sequencial
5,F23,82,2,Sequencial
6,F25,144,3,Sequencial
9,F33,357,5,Sequencial
10,F60,368,5,Sequencial


In [76]:
px.scatter(outliers_order_ranked_freqs, x="Diagnostico", y="Frequencia", size='Quantidade')

In [77]:
px.scatter(pd.concat([outliers_order_ranked_freqs, outliers_ranked_freqs]).sort_values(by = ['Frequencia', 'Quantidade']), x="Diagnostico", y="Frequencia", size='Quantidade', color='Label', title='Frequencia (Em Anos) de aparições de Transtornos por tipo de segmentação (Sequencial ou Geral)')

Pode-se notar que houve um sobreposição de todas as frequências, exceto a que ocorreu em um ano apenas (corte), o que indica que a alta de certos diagnósticos pode ter uma correlação com a sequência temporal

In [78]:
outliers_pares = outliers(contagem_agrupado(df, 'cid10_faixa', 'cid10_seg_faixa', 'Primeira_Faixa', 'Segunda_Faixa'))
# sns.relplot(outliers_pares, x='Ano', y='Quantidade', row='Primeira_Faixa', col='Segunda_Faixa')
px.scatter(outliers_pares, x='Ano', y='Quantidade', facet_row='Primeira_Faixa', facet_col='Segunda_Faixa', title='Distribuição de faixas entre Faixa de Primeiro Diagnóstico e Segundo Diagnóstico em Quantidade X Ano',height=650, width=1000)

TypeError: 'DataFrame' object is not callable

In [ ]:
outliers_pares = outliers(contagem_agrupado(df, 'cid10_faixa', 'cid10_faixa_alta', 'Primeira_Faixa', 'Primeira_Faixa_Alta'))
# sns.relplot(outliers_pares, x='Ano', y='Quantidade', row='Primeira_Faixa', col='Segunda_Faixa')
px.scatter(outliers_pares, x='Ano', y='Quantidade', facet_row='Primeira_Faixa', facet_col='Primeira_Faixa_Alta', title='Distribuição de faixas entre Faixa de Primeiro Diagnóstico e Primeiro Diagnóstico Alta em Quantidade X Ano',height=900, width=1100)

In [ ]:
outliers_pares = outliers(contagem_agrupado(df, 'cid10_seg_faixa', 'cid10_seg_faixa_alta', 'Segunda_Faixa', 'Segunda_Faixa_Alta'))
# sns.relplot(outliers_pares, x='Ano', y='Quantidade', row='Primeira_seg_Faixa', col='Segunda__Faixa')
px.scatter(outliers_pares, x='Ano', y='Quantidade', facet_col='Segunda_Faixa_Alta', facet_row='Segunda_Faixa', title='Distribuição de faixas entre Faixa de Segundo Diagnóstico e Segundo Diagnóstico Alta em Quantidade X Ano',height=1000, width=1100)

In [ ]:
outliers_pares = outliers(contagem_agrupado(df, 'cid10_seg_faixa_alta', 'cid10_faixa_alta', 'Segunda_Faixa_Alta', 'Primeira_Faixa_Alta'))
# sns.relplot(outliers_pares, x='Ano', y='Quantidade', row='Primeira_seg_Faixa', col='Segunda__Faixa')
px.scatter(outliers_pares, x='Ano', y='Quantidade', facet_col='Segunda_Faixa_Alta', facet_row='Primeira_Faixa_Alta', title='Distribuição de faixas entre Faixa de Primeiro Diagnóstico Alta e Segundo Diagnóstico Alta em Quantidade X Ano',height=1000, width=1200)

In [ ]:
outliers_pares = outliers(contagem_agrupado(df, 'cid10_diagnostico', 'cid10_seg_diagnostico', 'Primeiro_Diagnostico', 'Segundo_Diagnostico'))
# sns.relplot(outliers_pares, x='Ano', y='Quantidade', row='Primeira_seg_Faixa', col='Segunda__Faixa')
px.scatter(outliers_pares, x='Ano', y='Quantidade', color='Primeiro_Diagnostico', symbol='Segundo_Diagnostico', title='Distribuição de faixas entre Faixa de Primeiro Diagnóstico Alta e Segundo Diagnóstico Alta em Quantidade X Ano',height=1000, width=1200)

### Teste de independência entre diagnósticos

Como possuímos diagnósticos sobre o paciente (2 na entrada e 2 na alta), queremos ver se esses dados possuem algum tipo de associação entre si. 

Por se tratarem de dados categóricos é natural optar pelo teste Qui-Quadrado para independência, mais especificamente o teste de Pearson e o G-Test.

O teste de Pearson é o padrão de utilização para testes de independência, enquanto o G-Test é assumido para amostras grandes (3000+ no nosso caso).

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact

diagnosticos = ['cid10_diagnostico', 'cid10_seg_diagnostico', 'cid10_diagnostico_alta', 'cid10_seg_diagnostico_alta']

for i in range(len(diagnosticos)):
    for j in range(len(diagnosticos)):
        if j > i: 
            
            contingency_table = df.groupby([diagnosticos[i], diagnosticos[j]]).size().unstack(1).replace(np.nan, 0)
            res_chi= chi2_contingency(contingency_table)
            res_gtest = chi2_contingency(contingency_table, lambda_="log-likelihood")
            
            if res_chi.pvalue < 0.05:
                print(f'{diagnosticos[i]} e {diagnosticos[j]} apresentam uma associação significativa por Qui-Quadrado pois p = {res_chi.pvalue} < 0.05\n')
            else:
                print(f'{diagnosticos[i]} e {diagnosticos[j]} não apresentam uma associação significativa por Qui-Quadrado pois p = {res_chi.pvalue} > 0.05\n')
            
            
            if res_gtest.pvalue < 0.05:
                print(f'{diagnosticos[i]} e {diagnosticos[j]} apresentam uma associação significativa por Teste-G pois p = {res_gtest.pvalue} < 0.05\n')
            else:
                print(f'{diagnosticos[i]} e {diagnosticos[j]} não apresentam uma associação significativa por Teste-G pois p = {res_gtest.pvalue} > 0.05\n')
            
            print('')

cid10_diagnostico e cid10_seg_diagnostico apresentam uma associação significativa por Qui-Quadrado pois p = 0.0 < 0.05

cid10_diagnostico e cid10_seg_diagnostico não apresentam uma associação significativa por Teste-G pois p = 1.0 > 0.05


cid10_diagnostico e cid10_diagnostico_alta apresentam uma associação significativa por Qui-Quadrado pois p = 0.0 < 0.05

cid10_diagnostico e cid10_diagnostico_alta apresentam uma associação significativa por Teste-G pois p = 0.0 < 0.05


cid10_diagnostico e cid10_seg_diagnostico_alta apresentam uma associação significativa por Qui-Quadrado pois p = 0.0 < 0.05

cid10_diagnostico e cid10_seg_diagnostico_alta não apresentam uma associação significativa por Teste-G pois p = 1.0 > 0.05


cid10_seg_diagnostico e cid10_diagnostico_alta apresentam uma associação significativa por Qui-Quadrado pois p = 0.0 < 0.05

cid10_seg_diagnostico e cid10_diagnostico_alta não apresentam uma associação significativa por Teste-G pois p = 1.0 > 0.05


cid10_seg_diagnostico 

Devido a esparsialidade, faz-se necessário análisar a independência entre faixas de diagnóstico (F00-F09)

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
def teste_independencia(df, col1, col2):
    contingency_table = df.groupby([col1, col2]).size().unstack(1).replace(np.nan, 0)
    display(contingency_table)
    res_chi= chi2_contingency(contingency_table)
    res_gtest = chi2_contingency(contingency_table, lambda_="log-likelihood")
    
    if res_chi.pvalue < 0.05:
        print(f'{col1} e {col2} apresentam uma associação significativa por Qui-Quadrado pois p = {res_chi.pvalue} < 0.05\n')
    else:
        print(f'{col1} e {col2} não apresentam uma associação significativa por Qui-Quadrado pois p = {res_chi.pvalue} > 0.05\n')
    
    
    if res_gtest.pvalue < 0.05:
        print(f'{col1} e {col2} apresentam uma associação significativa por Teste-G pois p = {res_gtest.pvalue} < 0.05\n')
    else:
        print(f'{col1} e {col2} não apresentam uma associação significativa por Teste-G pois p = {res_gtest.pvalue} > 0.05\n')

In [ ]:
teste_independencia(df, 'cid10_faixa', 'cid10_seg_faixa')

cid10_seg_faixa,F01-F09,F10-F19,F20-F29,F30-F39,F40-F49,F50-F59,F60-F69,F70-F79,F80-F89,F90-F98,F99-F99
cid10_faixa,,,,,,,,,,,
F01-F09,8.0,15.0,18.0,7.0,0.0,0.0,5.0,22.0,0.0,0.0,0.0
F10-F19,42.0,378.0,194.0,231.0,35.0,0.0,185.0,94.0,0.0,24.0,2.0
F20-F29,46.0,394.0,159.0,122.0,21.0,1.0,55.0,73.0,1.0,0.0,0.0
F30-F39,23.0,276.0,94.0,66.0,35.0,4.0,328.0,13.0,0.0,6.0,0.0
F40-F49,2.0,3.0,4.0,7.0,5.0,0.0,10.0,0.0,0.0,0.0,0.0
F60-F69,1.0,57.0,7.0,112.0,19.0,0.0,17.0,9.0,0.0,0.0,0.0
F70-F79,18.0,26.0,28.0,9.0,4.0,2.0,8.0,2.0,1.0,4.0,0.0
F80-F89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
F90-F98,0.0,12.0,1.0,4.0,0.0,0.0,3.0,3.0,0.0,4.0,0.0


cid10_faixa e cid10_seg_faixa apresentam uma associação significativa por Qui-Quadrado pois p = 5.352082653692559e-146 < 0.05

cid10_faixa e cid10_seg_faixa apresentam uma associação significativa por Teste-G pois p = 1.1221402550233551e-119 < 0.05



In [ ]:
teste_independencia(df, 'cid10_faixa_alta', 'cid10_seg_faixa_alta')

cid10_seg_faixa_alta,F01-F09,F10-F19,F20-F29,F30-F39,F40-F49,F50-F59,F60-F69,F70-F79,F80-F89,F90-F98
cid10_faixa_alta,,,,,,,,,,
F01-F09,19.0,21.0,13.0,6.0,1.0,0.0,4.0,21.0,0.0,0.0
F10-F19,27.0,277.0,162.0,178.0,15.0,0.0,191.0,90.0,0.0,23.0
F20-F29,21.0,265.0,171.0,66.0,19.0,0.0,26.0,65.0,0.0,0.0
F30-F39,9.0,206.0,60.0,136.0,31.0,2.0,256.0,28.0,0.0,4.0
F40-F49,0.0,4.0,4.0,8.0,9.0,0.0,16.0,1.0,0.0,0.0
F50-F59,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
F60-F69,1.0,84.0,18.0,97.0,20.0,0.0,33.0,10.0,0.0,1.0
F70-F79,55.0,53.0,27.0,6.0,2.0,0.0,13.0,20.0,2.0,3.0
F80-F89,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0


cid10_faixa_alta e cid10_seg_faixa_alta apresentam uma associação significativa por Qui-Quadrado pois p = 5.3567491798628935e-213 < 0.05

cid10_faixa_alta e cid10_seg_faixa_alta apresentam uma associação significativa por Teste-G pois p = 4.1947005037687e-129 < 0.05



In [ ]:
teste_independencia(df, 'cid10_faixa', 'cid10_faixa_alta')

cid10_faixa_alta,F01-F09,F10-F19,F20-F29,F30-F39,F40-F49,F50-F59,F60-F69,F70-F79,F80-F89,F90-F98,F99-F99
cid10_faixa,,,,,,,,,,,
F01-F09,99.0,26.0,19.0,23.0,3.0,0.0,9.0,16.0,1.0,0.0,1.0
F10-F19,11.0,2618.0,117.0,92.0,2.0,0.0,35.0,32.0,1.0,32.0,0.0
F20-F29,68.0,246.0,1419.0,288.0,24.0,0.0,98.0,89.0,2.0,7.0,4.0
F30-F39,27.0,178.0,129.0,1406.0,34.0,3.0,211.0,22.0,0.0,4.0,2.0
F40-F49,1.0,3.0,7.0,14.0,26.0,0.0,15.0,1.0,0.0,0.0,0.0
F50-F59,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
F60-F69,4.0,50.0,9.0,79.0,15.0,0.0,246.0,5.0,0.0,3.0,0.0
F70-F79,11.0,18.0,16.0,10.0,1.0,0.0,5.0,126.0,2.0,3.0,0.0
F80-F89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0


cid10_faixa e cid10_faixa_alta apresentam uma associação significativa por Qui-Quadrado pois p = 0.0 < 0.05

cid10_faixa e cid10_faixa_alta apresentam uma associação significativa por Teste-G pois p = 0.0 < 0.05



In [ ]:
teste_independencia(df, 'cid10_seg_faixa', 'cid10_seg_faixa_alta')

cid10_seg_faixa_alta,F01-F09,F10-F19,F20-F29,F30-F39,F40-F49,F50-F59,F60-F69,F70-F79,F80-F89,F90-F98
cid10_seg_faixa,,,,,,,,,,
F01-F09,24.0,11.0,3.0,4.0,1.0,0.0,2.0,6.0,0.0,1.0
F10-F19,7.0,373.0,88.0,73.0,1.0,0.0,58.0,30.0,0.0,3.0
F20-F29,11.0,86.0,80.0,20.0,6.0,0.0,27.0,19.0,0.0,0.0
F30-F39,5.0,64.0,13.0,91.0,13.0,0.0,53.0,6.0,0.0,1.0
F40-F49,1.0,8.0,1.0,4.0,24.0,0.0,8.0,2.0,0.0,1.0
F50-F59,1.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0
F60-F69,2.0,43.0,19.0,55.0,9.0,0.0,157.0,12.0,1.0,3.0
F70-F79,17.0,26.0,15.0,5.0,0.0,0.0,4.0,67.0,0.0,0.0
F80-F89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


cid10_seg_faixa e cid10_seg_faixa_alta apresentam uma associação significativa por Qui-Quadrado pois p = 0.0 < 0.05

cid10_seg_faixa e cid10_seg_faixa_alta apresentam uma associação significativa por Teste-G pois p = 2.6996345813119578e-161 < 0.05

